In [2]:
__author__ = "Karol Saputa"
__copyright__ = "Copyright (C) 2004 Karol Saputa"
__license__ = "Public Domain"
__version__ = "1.0"

In [8]:
import csv

In [9]:
import pandas as pd

In [10]:
from pathlib import Path

In [11]:
def get_colnames(filename):
    layers = []
    colnames = []
    with open(filename) as f:
        for i in range(10):
            line = f.readline()
            if line[0] != '#':
                break
            elif i == 0:
                continue
            else:
                layers.append(line.split('=')[-1])

    for layer in layers:
        name = layer.split('|')[0].split('.')[-1]
        cols = layer.split('|')[1:]
        colnames.extend([name + '_' + c.strip() for c in cols])
        
    precols = ['s_num', 'ann_range', 'text']
    all_colnames = precols + colnames + ['']
    return all_colnames

In [12]:
def get_anns_cols(colnames):
     return [True if 'Regulative' in c  or 'Constitutive' in c else False for c in colnames]

In [13]:
def to_flatten(row, ann_cols):
    for i in range(len(row)):
        if ann_cols[i] and row[i] not in ['_', '*']:
            return True, i
    return False, None

In [18]:
def tsv2hr(filename):
    input = Path(filename)
    output = Path(input.parents[0], input.stem + '_hr.tsv')
    colnames = get_colnames(filename)
    p = pd.read_csv(filename, delimiter='\t', skiprows=5, header=None, comment='#')
    inds = []
    text = []
    ann_last = None
    is_first = True
    with open(output, 'w') as ff:
        ff.write('\t'.join(colnames) + '\n')
        for ind, row in p.iterrows():
            is_annotated, ann_ind = to_flatten(row, get_anns_cols(colnames))
            if is_annotated and row[ann_ind] != ann_last and not is_first:
                p.iloc[ind-1, 2] = ' '.join(text) # flatten last annotation
                ff.write('\t'.join([str(el) for el in p.iloc[ind-1]]) + '\n')
                is_first = True
                inds = [ind]
                ann_last = row[ann_ind]
                text = [row[2]]
            elif is_annotated:
                is_first = False
                ann_last = row[ann_ind]
                inds.append(ind)
                text.append(row[2])
            elif not is_annotated and not is_first:
                p.iloc[ind-1, 2] = ' '.join(text)
                ff.write('\t'.join([str(el) for el in p.iloc[ind-1]]) + '\n')
                is_first = True
                inds = [ind]
                ann_last = None
                text = []
            else:
                ff.write('\t'.join([str(el) for el in row]) + '\n') 
                is_first = True
                inds = []
                text = []

In [19]:
filename = 'data/doc_big.tsv'
tsv2hr(filename)